### custom loss

In [1]:
import tensorflow as tf

In [ ]:
loss = true - prediction

# 단순히 뻬기로 계산 하기 힘든 경우가 많다
setosa       0
versicolor   1
virsinica    2

# 그래서 one hot 인코딩 사용
setosa       0 0 1
versicolor   0 1 0
virsinica    1 0 0

In [ ]:
# loss 사용하는 3가지 방법
tf.keras.losses.SparseCategoricalCrossentropy
tf.keras.losses.sparse_categorical_crossentropy
'sparse_categorical_crossentropy'

In [ ]:
tf.keras.losses.huber # L1과 L2 를 합친 것 smoothL1 은 huber 를 변경 한 것

In [2]:
tf.__version__

'2.9.2'

In [ ]:
tensorflow 2.5 이상

In [ ]:
# subclass
- tf.keras.models.Model
- tf.keras.callbacks.Callback
- tf.keras.layers.Layer
- tf.keras.losses.Loss # __call__ funtion

# function
loss = tf.keras.losses.sparse_categorical_crossentropy

In [95]:
# loss = tf.keras.losses.SparseCategoricalCrossentropy()
loss = tf.keras.losses.sparse_categorical_crossentropy

In [96]:
loss

<function keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)>

### smooth L1

In [45]:
def smooth_l1(y_true, y_pred):
    error = y_true - y_pred
    is_one = tf.abs(error) < 1
    a = tf.abs(error) - 0.5
    b = tf.square(error) / 2
    return tf.where(is_one, b, a)

In [72]:
# 1번
def smooth_l1_thresh(y_true, y_pred, thresh=1):
    error = y_true - y_pred
    is_one = tf.abs(error) < thresh
    a = tf.abs(error) - 0.5
    b = tf.square(error) / 2
    return tf.where(is_one, b, a)

In [73]:
# 1번
from functools import partial
my_smooth_l1_thresh_partial = partial(smooth_l1_thresh, thresh=2)

In [81]:
# 2번 closure
def smooth_l1_thresh2(thresh):
    def smooth_l1_thresh(y_true, y_pred, thresh=1):
        error = y_true - y_pred
        is_one = tf.abs(error) < thresh
        a = tf.abs(error) - 0.5
        b = tf.square(error) / 2
        return tf.where(is_one, b, a)
    return smooth_l1_thresh

In [87]:
class SmoothL1(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
        pass
    def call(self, y_true, y_pred): # build
        error = y_true - y_pred
        is_one = tf.abs(error) < 1
        a = tf.abs(error) - 0.5
        b = tf.square(error) / 2
        return tf.where(is_one, b, a)

In [104]:
class SmoothL1Thresh(tf.keras.losses.Loss):
    def __init__(self, thresh=1):
        super().__init__()
        self.thresh = thresh
        
    def call(self, y_true, y_pred): # build
        error = y_true - y_pred
        is_one = tf.abs(error) < self.thresh
        a = tf.abs(error) - 0.5
        b = tf.square(error) / 2
        return tf.where(is_one, b, a)

In [105]:
(X_train, y_train),(X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [106]:
y_train = tf.keras.utils.to_categorical(y_train)

In [107]:
X_train = X_train / 255

In [108]:
input_ = tf.keras.Input((28,28))
x = tf.keras.layers.Flatten()(input_)
x = tf.keras.layers.Dense(10)(x)

In [109]:
model = tf.keras.Model(input_, x)

In [110]:
# model.compile(loss=smooth_l1)
# model.compile(loss=my_smooth_l1_thresh_partial) # 1번
# model.compile(loss=smooth_l1_thresh2(3)) # 2번 closure
# model.compile(loss= SmoothL1())
model.compile(loss= SmoothL1Thresh(3))

In [111]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
  42/1875 [..............................] - ETA: 4s - loss: 0.0705 

2022-08-12 22:24:14.220483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1875/1875 [==============================] - 5s 2ms/step - loss: 0.0252
Epoch 2/2
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0217


In [ ]:
# old style
- keras 자체가 tensorflow 전용이 아니였음
- 그래서 tensorflow, theano, cntk

In [112]:
import tensorflow.keras.backend as K

In [ ]:
# K <- framework 마다 맞춤형 함수로 변환

In [ ]:
class OldSmoothL1Thresh(tf.keras.losses.Loss):
    def __init__(self, thresh=1):
        super().__init__()
        self.thresh = thresh
        
    def call(self, y_true, y_pred): # build
        error = y_true - y_pred
        is_one = K.abs(error) < self.thresh
        a = K.abs(error) - 0.5
        b = K.square(error) / 2
        return tf.where(is_one, b, a)